In [8]:
# load necessary modules
import numpy as np 
from scipy.integrate import odeint
import os, sys 
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import time

# L96 system
def L96(u, F=10):
    u_new = np.zeros_like(u)
    for i in range(len(u)):
        u_new[i] = (u[(i + 1) % len(u)] - u[i - 2]) * u[i - 1] - u[i] + F
    return u_new

# single trajectory generator for L96
def generate_trajectory(state0, dt, n_steps):
    return odeint(lambda x, t: L96(x), state0, np.arange(0, n_steps*dt, dt))

# multiple trajectories generator for L96
@ut.timer
def generate_trajectories(num_trajectories, dim, dt, n_steps, save_folder, name):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    trajectories = np.zeros((num_trajectories, dim, n_steps))

    random_points =  np.random.normal(size=(num_trajectories, dim))
    generate = lambda *args: generate_trajectory(*args)[-1]
    states0 = Parallel(n_jobs=-1)(delayed(generate)(random_points[i], dt, int(1000/dt)) for i in range(num_trajectories))
    results = Parallel(n_jobs=-1)(delayed(generate_trajectory)(state0, dt, n_steps) for state0 in states0)
    for i in range(num_trajectories):
        trajectories[i, :, :] = results[i].T 
    np.save('{}/{}.npy'.format(save_folder, name), trajectories)
    return trajectories

In [3]:
seed = 43
np.random.seed(seed)
save_folder='../data/L96-trajectories'
dt = 0.01
n_steps = 200000
dim = 40
# find a point on the attractor
random_point =  np.random.normal(size=dim)
attractor_point = generate_trajectory(random_point, dt, n_steps=200000)[-1]
for i in [0, 1, 3, 7, 15]:
    train = generate_trajectory(attractor_point, dt/(i+1), n_steps)
    np.save('{}/{}.npy'.format(save_folder, f'train{i+1}'), train.T)

In [12]:
seed = 42
np.random.seed(seed)
num_trajectories = 1

states0 = []
start = time.time()
u_test = generate_trajectories(num_trajectories, dim, dt, 800*500, save_folder, 'test')
end = time.time()
print(f"Time taken to generate test data = {end-start:.2f}s")

Time taken by generate_trajectories is 22.2492 seconds
Time taken to generate test data = 22.25s


In [20]:
test = (u_test[0].T).reshape(500, -1, 40)
test = np.moveaxis(test, -1, -2)
np.random.shuffle(test)
np.save('{}/{}.npy'.format(save_folder, f'test'), test)
test.shape

(500, 40, 800)